In [2]:
import urllib.request, json 

# 5min BTC/USDT chart form Poloniex since 2017-01-01T00:00:00+00:00 till 2018-10-14T20:00:00+00:00
with urllib.request.urlopen("https://poloniex.com/public?command=returnChartData&currencyPair=USDT_BTC&start=1483228800&end=1539547200&period=300") as url:
    data = json.loads(url.read().decode())

In [3]:
print(data[0])

{'date': 1483228800, 'high': 965.00000066, 'low': 964.00000002, 'open': 965.00000055, 'close': 964.00000127, 'volume': 5398.83711674, 'quoteVolume': 5.59672254, 'weightedAverage': 964.64262399}


In [4]:
import csv
import numpy as np
import pandas as pd
 
df = pd.DataFrame(data=data, columns=['date', 'weightedAverage'])
        
df = df.fillna(method='ffill')
df.to_csv("poloniex_usdt_btc_20170101_weightedAverage_300.csv", sep=";", index=False)

df.head()

,date,weightedAverage
0,1483228800,964.642624
1,1483229100,965.858035
2,1483229400,963.946565
3,1483229700,962.024978
4,1483230000,963.568648


In [5]:
print(df.size)

375458


In [6]:
# Turn dataframe into a numpy floating point array
df_values = df.values
timestamps = df_values[:, 0]
dataset = df_values[:, 1].astype("float")

In [7]:
# Split the dataset into training (70%), cross validation (20%) and test (10%) set
samples_num = dataset.shape[0]

valid_split = 0.2
test_split = 0.1

dataset_train = dataset[0:int(samples_num * (1 - valid_split - test_split))]
dataset_valid = dataset[int(samples_num * (1 - valid_split - test_split)):int(samples_num * (1 - test_split))]
dataset_test = dataset[int(samples_num * (1 - test_split)):]

In [13]:
print(dataset_train.shape)
print(dataset_valid.shape)
print(dataset_test.shape)

(131410,)
(37546,)
(18773,)


In [17]:
print(dataset_train[:5])

[964.64262399 965.85803538 963.94656468 962.02497756 963.56864844]


In [12]:
# Makes a training examples from year data with a timesteps window size

def makeXy(data, timesteps):
    X = []
    y = []
    
    for i in range(timesteps, data.shape[0]):
        X.append(list(data[i-timesteps:i]))
        y.append(data[i])
        
    X, y = np.array(X), np.array(y)
    return X, y

In [53]:
print(np.array(dataset_train).shape)

(131410,)


In [52]:
print(np.array(dataset_train.shape).reshape(-1, 1))

[[131410]]


In [82]:
from sklearn.preprocessing import StandardScaler
from sklearn.externals import joblib

# Normalize data
scaler = StandardScaler().fit(np.array(dataset_train).reshape(-1, 1))
X_train = scaler.transform(np.array(dataset_train).reshape(-1, 1))
X_valid = scaler.transform(np.array(dataset_valid).reshape(-1, 1))
X_test = scaler.transform(np.array(dataset_test).reshape(-1, 1))

# Saving scaler for prediction
scaler_filename = "scaler.save"
joblib.dump(scaler, scaler_filename) 

print ("X_train:\n" + str(X_train))
print ("\nX_valid:\n" + str(X_valid))
print ("\nX_test:\n" + str(X_test))

X_train:
[[-0.95438328]
 [-0.9541113 ]
 [-0.95453904]
 ...
 [ 0.3791661 ]
 [ 0.37897781]
 [ 0.37332536]]

X_valid:
[[0.37374208]
 [0.37158819]
 [0.36742356]
 ...
 [0.2704622 ]
 [0.27149772]
 [0.27376986]]

X_test:
[[0.27597285]
 [0.27707803]
 [0.27824451]
 ...
 [0.26215537]
 [0.2623948 ]
 [0.26275016]]


In [76]:
import numpy as np

timestep = 20 # Make decision from last 20 days

# Create training examples

train_X, train_y = makeXy(X_train, timestep)
valid_X, valid_y = makeXy(X_valid, timestep)
test_X, test_y = makeXy(X_test, timestep)
    
print(train_X.shape)
print(train_y.shape)

train_X = train_X.reshape(train_X.shape[0],-1)
valid_X = valid_X.reshape(valid_X.shape[0],-1)
test_X = test_X.reshape(test_X.shape[0],-1)

print(train_X.shape)
print(train_y.shape)

(131390, 20, 1)
(131390, 1)
(131390, 20)
(131390, 1)


In [77]:
print(train_X[0])
print(train_y[0])

[-0.95438328 -0.9541113  -0.95453904 -0.95496905 -0.95462361 -0.95422342
 -0.95477771 -0.9545387  -0.9546288  -0.95445073 -0.95421625 -0.95449726
 -0.95449726 -0.95411198 -0.95449726 -0.95449726 -0.95411418 -0.95411198
 -0.95411198 -0.95410895]
[-0.95400951]


In [20]:
from keras.models import Sequential
from keras.layers.core import Dense, Activation
from keras.callbacks import Callback
from keras.optimizers import Adam
from keras.layers import Dropout
from keras import regularizers
from tensorflow import set_random_seed

# Set random seed to always predict the same values
np.random.seed(1)
set_random_seed(2) # Tensoflow

D:\ProgramFiles\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [38]:
# Saving loss and acc values

class TrainingHistory(Callback):

    def on_train_begin(self, logs={}):
        self.losses = []
        self.valid_losses = []
        self.accs = []
        self.valid_accs = []
        self.epoch = 0
    
    def on_epoch_end(self, epoch, logs={}):
        self.losses.append(logs.get('loss'))
        self.valid_losses.append(logs.get('val_loss'))
        self.accs.append(logs.get('acc'))
        self.valid_accs.append(logs.get('val_acc'))
        self.epoch += 1
            
history = TrainingHistory()

In [83]:
# Create model
model = Sequential()
model.add(Dense(200, activation='sigmoid', input_shape=(train_X.shape[1],)))
model.add(Dense(100, activation='sigmoid'))
model.add(Dense(100, activation='sigmoid'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='mean_squared_error',
              optimizer= Adam(),
              metrics=['accuracy'])

model.save('model.h5') # Saving model for prediction

In [87]:
# Setting up early stopping and weight saving on actually best solutions

from keras.callbacks import EarlyStopping
patience=30
early_stopping=EarlyStopping(patience=patience, verbose=1)

from keras.callbacks import ModelCheckpoint
checkpointer=ModelCheckpoint(filepath='weights.hdf5', save_best_only=True, verbose=1)

In [89]:
# Training model
model.fit(train_X[::timestep], train_y[::timestep],
          batch_size=4,
          epochs=2500,
          verbose=2, 
          validation_data=(valid_X[::timestep],valid_y[::timestep]), 
          callbacks=[checkpointer, early_stopping, history], 
          shuffle= True)

Train on 6570 samples, validate on 1877 samples
Epoch 1/2500
 - 1s - loss: 0.4808 - acc: 0.0000e+00 - val_loss: 4.0635e-04 - val_acc: 0.0000e+00

Epoch 00001: val_loss did not improve from 0.00006
Epoch 2/2500
 - 1s - loss: 0.4808 - acc: 0.0000e+00 - val_loss: 1.6560e-04 - val_acc: 0.0000e+00

Epoch 00002: val_loss did not improve from 0.00006
Epoch 3/2500
 - 1s - loss: 0.4808 - acc: 0.0000e+00 - val_loss: 1.2614e-04 - val_acc: 0.0000e+00

Epoch 00003: val_loss did not improve from 0.00006
Epoch 4/2500
 - 1s - loss: 0.4808 - acc: 0.0000e+00 - val_loss: 8.9035e-05 - val_acc: 0.0000e+00

Epoch 00004: val_loss did not improve from 0.00006
Epoch 5/2500
 - 1s - loss: 0.4808 - acc: 0.0000e+00 - val_loss: 2.8952e-04 - val_acc: 0.0000e+00

Epoch 00005: val_loss did not improve from 0.00006
Epoch 6/2500
 - 1s - loss: 0.4808 - acc: 0.0000e+00 - val_loss: 7.7827e-05 - val_acc: 0.0000e+00

Epoch 00006: val_loss did not improve from 0.00006
Epoch 7/2500
 - 1s - loss: 0.4808 - acc: 0.0000e+00 - val_


Epoch 00054: val_loss did not improve from 0.00004
Epoch 55/2500
 - 1s - loss: 0.4808 - acc: 0.0000e+00 - val_loss: 5.1237e-05 - val_acc: 0.0000e+00

Epoch 00055: val_loss did not improve from 0.00004
Epoch 56/2500
 - 1s - loss: 0.4808 - acc: 0.0000e+00 - val_loss: 5.3891e-05 - val_acc: 0.0000e+00

Epoch 00056: val_loss did not improve from 0.00004
Epoch 57/2500
 - 1s - loss: 0.4808 - acc: 0.0000e+00 - val_loss: 4.9435e-05 - val_acc: 0.0000e+00

Epoch 00057: val_loss did not improve from 0.00004
Epoch 58/2500
 - 1s - loss: 0.4808 - acc: 0.0000e+00 - val_loss: 9.1895e-05 - val_acc: 0.0000e+00

Epoch 00058: val_loss did not improve from 0.00004
Epoch 59/2500
 - 1s - loss: 0.4808 - acc: 0.0000e+00 - val_loss: 4.5203e-05 - val_acc: 0.0000e+00

Epoch 00059: val_loss did not improve from 0.00004
Epoch 60/2500
 - 1s - loss: 0.4808 - acc: 0.0000e+00 - val_loss: 6.4928e-05 - val_acc: 0.0000e+00

Epoch 00060: val_loss did not improve from 0.00004
Epoch 61/2500
 - 1s - loss: 0.4808 - acc: 0.0000

 - 1s - loss: 0.4808 - acc: 0.0000e+00 - val_loss: 4.2408e-05 - val_acc: 0.0000e+00

Epoch 00109: val_loss did not improve from 0.00003
Epoch 110/2500
 - 1s - loss: 0.4808 - acc: 0.0000e+00 - val_loss: 4.9194e-05 - val_acc: 0.0000e+00

Epoch 00110: val_loss did not improve from 0.00003
Epoch 111/2500
 - 1s - loss: 0.4808 - acc: 0.0000e+00 - val_loss: 5.0508e-05 - val_acc: 0.0000e+00

Epoch 00111: val_loss did not improve from 0.00003
Epoch 112/2500
 - 1s - loss: 0.4808 - acc: 0.0000e+00 - val_loss: 4.7706e-05 - val_acc: 0.0000e+00

Epoch 00112: val_loss did not improve from 0.00003
Epoch 113/2500
 - 1s - loss: 0.4808 - acc: 0.0000e+00 - val_loss: 4.7224e-05 - val_acc: 0.0000e+00

Epoch 00113: val_loss did not improve from 0.00003
Epoch 114/2500
 - 1s - loss: 0.4808 - acc: 0.0000e+00 - val_loss: 6.5333e-05 - val_acc: 0.0000e+00

Epoch 00114: val_loss did not improve from 0.00003
Epoch 115/2500
 - 1s - loss: 0.4808 - acc: 0.0000e+00 - val_loss: 3.5965e-05 - val_acc: 0.0000e+00

Epoch 001